# NNIA: Tutorial 5 - 00.12.2017

In [1]:
# Set notebook to full width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Organizational things
### General
* How was the quiz?

## Assignment-5

#### 1. [Practicing Backpropagation](http://cs231n.github.io/optimization-2/#grad)

Given the following function $$ f(x, y) = \frac{x + \sigma(y)}{\sigma(x) + (x+y)^2}$$ with $\sigma(x) = \frac{1}{1+e^{-x}}$ and $x, y \in \mathbb{R}$.

In [36]:
import math

# Define primitives

def add(x, y):
    return x + y

def mult(x, y):
    return x * y

def square(x):
    return x ** 2

def exp(x):
    return math.exp(x)

def divide(x):
    return 1 / x

def sigmoid(x):
    return divide(add(1, exp(mult(-1, x))))

In [39]:
# Define local derivatives of the primitives

def dadd(x, y):
    return 1.0

def dmult(x, y, dx, dy):
    return x * dy + y * dx

def dmult_left(x, y):
    return y

def dmult_right(x, y):
    return x

def dsquare(x):
    return 2 * x

def dexp(x):
    return exp(x)

def ddivide(x):
    return -1.0 * (1.0 / x ** 2)

def dsigmoid(x):
    return sigmoid(x) * (1.0 - sigmoid(x))

In [28]:
# Compute forward pass for specific values of x and y
x = 2
y = 3

def forward(x, y):
    m = {}
    
    # Numerator
    m['0'] = sigmoid(y) # sigmoid(y)
    m['1'] = add(x, m['0']) # numerator
    
    # Denominator
    m['2'] = sigmoid(x) # sigmoid(x)
    m['3'] = add(x, y) # (x + y)
    m['4'] = square(m['3']) # (x + y)^2
    m['5'] = add(m['2'], m['4']) # denominator
    
    # f
    m['6'] = divide(m['5']) # 1 / denominator
    m['7'] = mult(m['1'], m['6']) # numerator * denominator
    
    return m

evaluate_forward = forward(x, y)
print(evaluate_forward)

{'0': 0.9525741268224334, '1': 2.9525741268224333, '2': 0.8807970779778823, '3': 5, '4': 25, '5': 25.88079707797788, '6': 0.03863868631970789, '7': 0.11408358552197742}


In [81]:
# Compute backward pass

def backward(x, y):
    
    f = forward(x, y)
    m = {}
    
    m['0'] = 1.0 # df/df
    
    # df/dnum
    m['1'] = m['0'] * f['6'] # df/d1 = df/df * df/d1 
    m['2'] = m['1'] * 1.0 # df/dx = df/df * df/d1 * d1/dx
    m['3'] = m['1'] * 1.0 # df/d0 = df/df * df/d1 * d1/d0
    m['4'] = m['3'] * 1.0 * (sigmoid(y) * (1 - sigmoid(y))) # df/dy = df/df * df/d1 * d1/d0 * d0/dy
    
    # df/ddom
    m['5'] = m['0'] * f['1'] # df/d6 = df/df * df/d6
    m['6'] = m['5'] * (-1.0 * (1.0 / f['5'] ** 2)) # df/d5 = df/df * df/d6 * d6/d5
    m['7'] = m['6'] * 1.0  # df/d2 = df/df * df/d6 * d6/d5 * d5/d2
    m['8'] = m['6'] * 1.0  # df/d4 = df/df * df/d6 * d6/d5 * d5/d4
    m['9'] = m['7'] * (sigmoid(x) * (1.0 - sigmoid(x)))  # df/dx = df/df * df/d6 * d6/d5 * d5/d2 * d2/dx
    m['10'] = m['8'] * 2 * f['3'] # df/d3 = df/df * df/d6 * d6/d5 * d5/d4 * d4/d3
    m['11'] = m['10'] * 1.0 # df/dx = df/df * df/d6 * d6/d5 * d5/d4 * d4/d3 * d3/dx
    m['12'] = m['10'] * 1.0 # df/dy = df/df * df/d6 * d6/d5 * d5/d4 * d4/d3 * d3/dy
    
    # Add up results for df/dx and df/dy
    m['13'] = m['2'] + m['9'] + m['11']
    m['14'] = m['4'] + m['12']
    
    return m

evaluate_backward = backward(x, y)
print(evaluate_backward)

{'0': 1.0, '1': 0.03863868631970789, '2': 0.03863868631970789, '3': 0.03863868631970789, '4': 0.0017455667843148878, '5': 2.9525741268224333, '6': -0.004408039875211255, '7': -0.004408039875211255, '8': -0.004408039875211255, '9': -0.00046281591110005553, '10': -0.044080398752112546, '11': -0.044080398752112546, '12': -0.044080398752112546, '13': -0.00590452834350471, '14': -0.04233483196779766}


In [82]:
# Compute backward pass

def backward2(x, y):
    
    f = forward(x, y)
    m = {}
    
    m['0'] = 1.0 # df/df
    
    # df/dnum
    m['1'] = m['0'] * f['6'] # df/d1 = df/df * df/d1 
    m['2'] = m['1'] * 1.0 # df/dx = df/df * df/d1 * d1/dx
    m['3'] = m['1'] * 1.0 # df/d0 = df/df * df/d1 * d1/d0
    m['4'] = m['3'] * 1.0 * (f['0'] * (1 - f['0'])) # df/dy = df/df * df/d1 * d1/d0 * d0/dy
    
    # df/ddom
    m['5'] = m['0'] * f['1'] # df/d6 = df/df * df/d6
    m['6'] = m['5'] * (-1.0 * (1.0 / f['5'] ** 2)) # df/d5 = df/df * df/d6 * d6/d5
    m['7'] = m['6'] * 1.0  # df/d2 = df/df * df/d6 * d6/d5 * d5/d2
    m['8'] = m['6'] * 1.0  # df/d4 = df/df * df/d6 * d6/d5 * d5/d4
    m['9'] = m['7'] * (f['2'] * (1 - f['2']))  # df/dx = df/df * df/d6 * d6/d5 * d5/d2 * d2/dx
    m['10'] = m['8'] * 2 * f['3'] # df/d3 = df/df * df/d6 * d6/d5 * d5/d4 * d4/d3
    m['11'] = m['10'] * 1.0 # df/dx = df/df * df/d6 * d6/d5 * d5/d4 * d4/d3 * d3/dx
    m['12'] = m['10'] * 1.0 # df/dy = df/df * df/d6 * d6/d5 * d5/d4 * d4/d3 * d3/dy
    
    # Add up results for df/dx and df/dy
    m['13'] = m['2'] + m['9'] + m['11']
    m['14'] = m['4'] + m['12']
    
    
    return m

evaluate_backward2 = backward2(x, y)
print(evaluate_backward)

{'0': 1.0, '1': 0.03863868631970789, '2': 0.03863868631970789, '3': 0.03863868631970789, '4': 0.0017455667843148878, '5': 2.9525741268224333, '6': -0.004408039875211255, '7': -0.004408039875211255, '8': -0.004408039875211255, '9': -0.00046281591110005553, '10': -0.044080398752112546, '11': -0.044080398752112546, '12': -0.044080398752112546, '13': -0.00590452834350471, '14': -0.04233483196779766}


In [79]:
# Compute backward pass

def backward3(x, y):
    
    f = forward(x, y)
    m = {}
    
    m['0'] = 1.0 # df/df
    
    # df/dnum
    m['1'] = m['0'] * dmult_left(f['1'], f['6']) # df/d1 = df/df * df/d1 
    m['2'] = m['1'] * dadd(f['2'], f['4']) # df/dx = df/df * df/d1 * d1/dx
    m['3'] = m['1'] * dadd(f['2'], f['4']) # df/d0 = df/df * df/d1 * d1/d0
    m['4'] = m['3'] * dsigmoid(y) # df/dy = df/df * df/d1 * d1/d0 * d0/dy
    
    # df/ddom
    m['5'] = m['0'] * dmult_right(f['1'], f['6']) # df/d6 = df/df * df/d6
    m['6'] = m['5'] * ddivide(f['5']) # df/d5 = df/df * df/d6 * d6/d5
    m['7'] = m['6'] * dadd(f['2'], f['4']) # df/d2 = df/df * df/d6 * d6/d5 * d5/d2
    m['8'] = m['6'] * dadd(f['2'], f['4']) # df/d4 = df/df * df/d6 * d6/d5 * d5/d4
    m['9'] = m['7'] * dsigmoid(x) # df/dx = df/df * df/d6 * d6/d5 * d5/d2 * d2/dx
    m['10'] = m['8'] * dsquare(f['3']) # df/d3 = df/df * df/d6 * d6/d5 * d5/d4 * d4/d3
    m['11'] = m['10'] * dadd(x, y) # df/dx = df/df * df/d6 * d6/d5 * d5/d4 * d4/d3 * d3/dx
    m['12'] = m['10'] * dadd(x, y) # df/dy = df/df * df/d6 * d6/d5 * d5/d4 * d4/d3 * d3/dy
    
    # Add up results for df/dx and df/dy
    m['13'] = m['2'] + m['9'] + m['11']
    m['14'] = m['4'] + m['12']
    
    return m

evaluate_backward3 = backward3(x, y)
print(evaluate_backward)

(-0.00590452834350471, -0.04233483196779766)


#### 2. Feed-forward Neural Network in Tensorflow

## Assignment-6: Questions?